In [160]:
import pandas as pd

df = pd.read_csv("../notebooks/data/Ana Health_Tabela Modelo Previsão Churn - Tabela.csv")

In [161]:
from data_preprocessing import preprocessing,mergeHeader_Columns
from data_feature_engineering import feature_engineering

In [162]:
data_merge = mergeHeader_Columns(df).iloc[50]

data_dataframe = data_merge.to_json( force_ascii=False)

In [163]:
data_dataframe

'{"PESSOA_PIPEDRIVE_id_person":"1080","PESSOA_PIPEDRIVE_birthdate":"1991-11-20","PESSOA_PIPEDRIVE_id_gender":"63","PESSOA_PIPEDRIVE_id_marrital_status":"80","PESSOA_PIPEDRIVE_state":"Pernambuco","PESSOA_PIPEDRIVE_city":"Recife","PESSOA_PIPEDRIVE_postal_code":"50070-045","PESSOA_PIPEDRIVE_id_health_plan":null,"PESSOA_PIPEDRIVE_id_person_recommendation":null,"PESSOA_PIPEDRIVE_contract_start_date":"2021-07-19","PESSOA_PIPEDRIVE_contract_end_date":"2023-08-18","PESSOA_PIPEDRIVE_id_continuity_pf":null,"PESSOA_PIPEDRIVE_Canal de Preferência":null,"PESSOA_PIPEDRIVE_notes_count":"5","PESSOA_PIPEDRIVE_done_activities_count":"50","PESSOA_PIPEDRIVE_Recebe Comunicados?":null,"PESSOA_PIPEDRIVE_Interesses":null,"PESSOA_PIPEDRIVE_Pontos de Atenção":null,"FUNIL_ASSINATURA_PIPEDRIVE_id_stage":"64","FUNIL_ASSINATURA_PIPEDRIVE_id_org":null,"FUNIL_ASSINATURA_PIPEDRIVE_status":"lost","FUNIL_ASSINATURA_PIPEDRIVE_start_of_service":"2021-07-19","FUNIL_ASSINATURA_PIPEDRIVE_lost_time":"2023-07-26 12:59:22","FUN

In [164]:
data_dataframe = pd.DataFrame([data_merge])

In [165]:
data_dataframe.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_birthdate,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_id_health_plan,PESSOA_PIPEDRIVE_id_person_recommendation,PESSOA_PIPEDRIVE_contract_start_date,...,TWILIO_Ligações Outbound,TWILIO_Data Última Ligações Outbound,COBRANÇA_VINDI_Qde Total de Faturas,COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança,COBRANÇA_VINDI_Método de Pagamento,COBRANÇA_VINDI_Valor Médio da Mensalidade,COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento,COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes,COBRANÇA_VINDI_Valor Total Inadimplência,COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos
51,1080,1991-11-20,63,80,Pernambuco,Recife,50070-045,NaN,NaN,2021-07-19,...,12,2023-07-14 20:55:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
data_dataframe.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_birthdate,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_id_health_plan,PESSOA_PIPEDRIVE_id_person_recommendation,PESSOA_PIPEDRIVE_contract_start_date,...,TWILIO_Ligações Outbound,TWILIO_Data Última Ligações Outbound,COBRANÇA_VINDI_Qde Total de Faturas,COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança,COBRANÇA_VINDI_Método de Pagamento,COBRANÇA_VINDI_Valor Médio da Mensalidade,COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento,COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes,COBRANÇA_VINDI_Valor Total Inadimplência,COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos
51,1080,1991-11-20,63,80,Pernambuco,Recife,50070-045,NaN,NaN,2021-07-19,...,12,2023-07-14 20:55:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
data_dataframe["PESSOA_PIPEDRIVE_id_health_plan"]

51   NaN
Name: PESSOA_PIPEDRIVE_id_health_plan, dtype: float64

In [168]:
data_dataframe["PESSOA_PIPEDRIVE_id_person_recommendation"]

51   NaN
Name: PESSOA_PIPEDRIVE_id_person_recommendation, dtype: float64

In [169]:
import datetime as dt
from datetime import datetime
from sklearn.impute import SimpleImputer
import numpy as np

data_dataframe = data_dataframe.drop(["PESSOA_PIPEDRIVE_id_person_recommendation","PESSOA_PIPEDRIVE_Recebe Comunicados?", "PESSOA_PIPEDRIVE_Interesses", "PESSOA_PIPEDRIVE_Pontos de Atenção", "FUNIL_ONBOARDING_PIPEDRIVE_id_label"], axis=1)

data_dataframe['PESSOA_PIPEDRIVE_birthdate'] = pd.to_datetime(data_dataframe['PESSOA_PIPEDRIVE_birthdate'])
data_dataframe['PESSOA_PIPEDRIVE_age'] = data_dataframe['PESSOA_PIPEDRIVE_birthdate'].apply(
    lambda x: datetime.today().year - x.year - ((datetime.today().month, datetime.today().day) < (x.month, x.day))
)

imputer = SimpleImputer(strategy='mean')
data_dataframe['PESSOA_PIPEDRIVE_age'] = imputer.fit_transform(data_dataframe['PESSOA_PIPEDRIVE_age'].values.reshape(-1, 1))
data_dataframe['PESSOA_PIPEDRIVE_age'] = np.round(data_dataframe['PESSOA_PIPEDRIVE_age']).astype(int)

data_dataframe = data_dataframe.drop(['PESSOA_PIPEDRIVE_birthdate'], axis=1 )

rows_to_drop = data_dataframe[data_dataframe["PESSOA_PIPEDRIVE_id_gender"].isin([117,110,111])]
data_dataframe = data_dataframe.drop(rows_to_drop.index, axis=0)
data_dataframe["PESSOA_PIPEDRIVE_id_gender"].fillna(64, inplace=True)
data_dataframe["PESSOA_PIPEDRIVE_id_marrital_status"].fillna(80, inplace=True)


data_dataframe["PESSOA_PIPEDRIVE_state"].fillna(data_dataframe["PESSOA_PIPEDRIVE_state"].mode()[0], inplace=True)


data_dataframe["PESSOA_PIPEDRIVE_city"].fillna(data_dataframe["PESSOA_PIPEDRIVE_city"].mode()[0], inplace=True)

def convert_to_int(x):
    if pd.notna(x) and int(x) == 412:
        return 1
    else:
        return 0


mode_value = data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].mode()
if not mode_value.empty:
    data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].fillna(mode_value[0], inplace=True)
data_dataframe['PESSOA_PIPEDRIVE_has_public_health_plan'] = data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].apply(convert_to_int)
data_dataframe = data_dataframe.drop(['PESSOA_PIPEDRIVE_id_health_plan'], axis=1)
data_dataframe["PESSOA_PIPEDRIVE_tem_data_dataframe_de_termino_de_contrato"] = data_dataframe["PESSOA_PIPEDRIVE_contract_end_date"].apply(lambda x: 0 if pd.isna(x) else 1)
data_dataframe.drop(["PESSOA_PIPEDRIVE_id_continuity_pf"], axis=1, inplace=True)
data_dataframe["PESSOA_PIPEDRIVE_Canal de Preferência"].fillna(0, inplace=True)
data_dataframe["PESSOA_PIPEDRIVE_Tem_Canal_de_Preferência"] = data_dataframe["PESSOA_PIPEDRIVE_Canal de Preferência"].apply(lambda x: 1 if int(x) > 0  else 0)
data_dataframe["PESSOA_PIPEDRIVE_has_notes"] = data_dataframe["PESSOA_PIPEDRIVE_notes_count"].apply(lambda x: 1 if int(x) > 0 else 0)

# preprocessing 2
data_dataframe = data_dataframe.drop(["ATENDIMENTOS_AGENDA_Faltas Psicoterapia","TWILIO_Ligações Inbound", "TWILIO_Data Última Ligações Inbound","COBRANÇA_VINDI_Qde Total de Faturas","COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança","COBRANÇA_VINDI_Método de Pagamento","COBRANÇA_VINDI_Valor Médio da Mensalidade","COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento","COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes","COBRANÇA_VINDI_Valor Total Inadimplência"], axis=1)

data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].fillna(0, inplace=True)



data_dataframe = data_dataframe.drop(columns="WHOQOL_Qde Respostas WHOQOL")

data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico"].str.split(';').str[-1].str.strip()

data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].str.replace(",",".")

data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].astype(float) 

data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].fillna(data_dataframe["WHOQOL_Físico_New"].median())

data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico"].str.split(';').str[-1].str.strip()

data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].str.replace(",",".")

data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].astype(float) 

data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].fillna(data_dataframe["WHOQOL_Psicológico_New"].median())

data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social"].str.split(';').str[-1].str.strip()

data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].str.replace(",",".")

data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].astype(float) 

data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].fillna(data_dataframe["WHOQOL_Social_New"].median())

data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental"].str.split(';').str[-1].str.strip()

data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].str.replace(",",".")

data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].astype(float) 

data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].fillna(data_dataframe["WHOQOL_Ambiental_New"].median())

data_dataframe["COMUNICARE_Problemas Abertos Bool"] = data_dataframe["COMUNICARE_Problemas Abertos"].notnull().astype(int)

data_dataframe["COMUNICARE_Problemas Abertos Bool"].fillna(0, inplace=True)

data_dataframe["TWILIO_Mensagens Inbound"].fillna(0, inplace=True)

data_dataframe["TWILIO_Data Última Mensagens Inbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Mensagens Inbound"])

data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Mensagens Inbound"]

data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].fillna('', inplace=True)

data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].astype(str)

data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].str.extract('(\d+) days').astype(float)

data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] < data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].median()
data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].fillna(0)
data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].replace(True, 1, inplace=True)
data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].replace(False, 0, inplace=True)

data_dataframe = data_dataframe.drop(columns=["WHOQOL_Ambiental","WHOQOL_Social","WHOQOL_Físico","WHOQOL_Psicológico","COMUNICARE_Problemas Abertos","TWILIO_Data Última Mensagens Inbound","ATENDIMENTOS_AGENDA_Datas Psicoterapia","TWILIO_Data Última Mensagens Inbound Tempo Passado"])

# preprocessing 3
for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
    if pd.notna(valor) == False: 
        if pd.notna(data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_end_date"]):
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_end_date"]

data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].fillna(dt.date.today(), inplace=True)

for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
    index = str(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"]).find(";")
    if index != -1:
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"][:index]

for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
    tamanho = len(str(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"]))
    if tamanho > 10:
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"][:10]	

for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
    data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = pd.to_datetime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], format='%Y-%m-%d', errors='coerce')
    data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].strftime('%Y-%m-%d')

tempo_permanencia = []

for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].items():
    if pd.notna(valor):
        index = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].find(";")
        if index != -1:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"][:index]

for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
    if pd.notna(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"]):
        tempo_1 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
        tempo_2 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"], '%Y-%m-%d')
        tempo_permanencia.append(str(tempo_1 - tempo_2))
    else:
        tempo_1 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
        tempo_2 = datetime.strptime(data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_start_date"], '%Y-%m-%d')
        tempo_permanencia.append(str(tempo_1 - tempo_2))

data_dataframe['stay_time'] = tempo_permanencia

for indice, valor in data_dataframe["stay_time"].items():
    index = data_dataframe.loc[indice, "stay_time"].find(",")
    if index != -1:
        data_dataframe.loc[indice, "stay_time"] = data_dataframe.loc[indice, "stay_time"][:index]

for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].items():
    if pd.notna(valor):  
        index = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].find(";")
        if index != -1: 
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"][:index]
    else:
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = "Assinatura ativa"
        
    contagem = data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].value_counts()

    agrupamento = contagem[contagem < 20].index
    data_dataframe.loc[data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].isin(agrupamento), "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = "Outro"

data_dataframe['FUNIL_ONBOARDING_PIPEDRIVE_add_time'].fillna('Não iniciado', inplace=True)

ultimos_estados = []

for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
    if valor != "Não iniciado":
        if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome"]):
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting"]):
                if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"]):
                    ultimos_estados.append("Questionário")
                else:
                    ultimos_estados.append("Primeira reunião")
            else:
                ultimos_estados.append("Boas-vindas")
        
    else:
        ultimos_estados.append("Não iniciado")

data_dataframe["last_stage_concluded"] = ultimos_estados

data_dataframe.drop(["FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"], axis=1, inplace=True)

data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_status"].fillna("Não iniciado", inplace=True)

tempo = []

for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
    if valor != "Não iniciado":
        if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"]) == True:
            tempo.append(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"])
        elif pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"]) == True:
            tempo.append(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"])
        else:
            tempo.append("Em aberto")
    else:
        tempo.append("Não iniciado")

data_dataframe['process_time'] = tempo

data_dataframe.drop(["FUNIL_ONBOARDING_PIPEDRIVE_won_time", "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"], axis=1, inplace=True)

data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_activities_count"] = data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_activities_count"].fillna(0)

data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimento Médico"].fillna(0)

data_dataframe["ATENDIMENTOS_AGENDA_Faltas Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Faltas Atendimento Médico"].fillna(0)

data_dataframe["ATENDIMENTOS_AGENDA_Datas Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Datas Atendimento Médico"].fillna("Nunca ocorreu")

for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].items():
    if pd.notna(valor) == False:
        if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_status"]) == False and pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_add_time"]):
            data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo em aberto"
        if data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_status"] == "won":
            data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo concluído"
        else:
            data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo não iniciado"

    contagem = data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].value_counts()

    agrupamento = contagem[contagem < 23].index
    data_dataframe.loc[data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].isin(agrupamento), "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Outro"

data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"].fillna(0)

data_dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"].fillna(0)

data_dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"].fillna("Nunca ocorreu")

data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].fillna(0)

# preprocessing 4

data_dataframe["TWILIO_Mensagens Outbound"].fillna(0, inplace=True)

data_dataframe["TWILIO_Data Última Mensagens Outbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Mensagens Outbound"])

data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Mensagens Outbound"]

data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].fillna('', inplace=True)

data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].astype(str)

data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].str.extract('(\d+) days').astype(float)

data_dataframe = data_dataframe.drop(["TWILIO_Data Última Mensagens Outbound"], axis=1)

data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] < data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].median()
data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].fillna(0)
data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].replace(True, 1, inplace=True)
data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].replace(False, 0, inplace=True)

data_dataframe = data_dataframe.drop(["TWILIO_Data Última Mensagens Outbound Tempo passado"], axis=1)

data_dataframe["TWILIO_Ligações Outbound"].fillna(0, inplace=True)

data_dataframe["TWILIO_Data Última Ligações Outbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Ligações Outbound"])

data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Ligações Outbound"]

data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].fillna('', inplace=True)

data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].astype(str)

data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].str.extract('(\d+) days').astype(float)

data_dataframe = data_dataframe.drop(["TWILIO_Data Última Ligações Outbound"], axis=1)

data_dataframe["TWILIO_Data Última Ligações Outbound Recente"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] < data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].median()
data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].fillna(0)
data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].replace(True, 1, inplace=True)
data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].replace(False, 0, inplace=True)

data_dataframe = data_dataframe.drop(["TWILIO_Data Última Ligações Outbound Tempo passado"], axis=1)

data_dataframe = data_dataframe.drop(["COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos"], axis=1)

# preprocessing especials 

data_dataframe = data_dataframe[data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'].isin(['won', 'lost'])]

data_dataframe = data_dataframe.drop(data_dataframe[(data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'] == 'won') & (~data_dataframe['PESSOA_PIPEDRIVE_contract_end_date'].isnull())].index)

data_dataframe = data_dataframe.drop(data_dataframe[(data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'] == 'lost') & (data_dataframe['PESSOA_PIPEDRIVE_contract_end_date'].isnull())].index)

#exportando df pronto

data_dataframe.to_csv('../notebooks/data/data-preprocessed.csv', index=False)


/var/folders/5b/y5xgth0n79lbt3pkmgmjphzh0000gn/T/ipykernel_16215/3407921406.py:226: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Processo concluído' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo concluído"


In [170]:
data_dataframe.shape

(1, 51)

In [171]:
data_dataframe["WHOQOL_Físico_New"]

51    2.9
Name: WHOQOL_Físico_New, dtype: float64

In [172]:
import pandas as pd
import numpy as np
from category_encoders import CountEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy import stats
from sklearn.utils import resample
from sqlalchemy import false

dataframe = pd.read_csv("../notebooks/data/data-preprocessed.csv")


dataframe["stay_time"] = dataframe["stay_time"].str.extract('(\d+) days').astype(float)

dataframe["stay_time"].fillna(0,inplace=True)

dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"].fillna(0,inplace=True)
dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"].fillna(0,inplace=True)
dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"].fillna("Nunca ocorreu",inplace=True)

# dataframe = dataframe[~dataframe['ATENDIMENTOS_AGENDA_Datas Acolhimento'].astype(str).str.contains("Nunca ocorreu")]

# dataframe = dataframe.explode('ATENDIMENTOS_AGENDA_Datas Acolhimento')

dataframe["TWILIO_Mensagens Já Enviou"] = dataframe["TWILIO_Mensagens Inbound"] > 0
dataframe["TWILIO_Mensagens Razão"] = dataframe["TWILIO_Mensagens Outbound"] / dataframe["TWILIO_Mensagens Inbound"].where(dataframe["TWILIO_Mensagens Já Enviou"], 1)

dataframe["PESSOA_PIPEDRIVE CRIANÇA"] = dataframe["PESSOA_PIPEDRIVE_age"] <= 16
dataframe["PESSOA_PIPEDRIVE CRIANÇA"].fillna(0)
dataframe["PESSOA_PIPEDRIVE CRIANÇA"].replace(True, 1, inplace=True)
dataframe["PESSOA_PIPEDRIVE CRIANÇA"].replace(False, 0, inplace=True)

dataframe["PESSOA_PIPEDRIVE JOVEM"] = (dataframe["PESSOA_PIPEDRIVE_age"] > 16) & (dataframe["PESSOA_PIPEDRIVE_age"] <= 30)
dataframe["PESSOA_PIPEDRIVE JOVEM"].fillna(0)
dataframe["PESSOA_PIPEDRIVE JOVEM"].replace(True, 1, inplace=True)
dataframe["PESSOA_PIPEDRIVE JOVEM"].replace(False, 0, inplace=True)

dataframe["PESSOA_PIPEDRIVE ADULTO"] = (dataframe["PESSOA_PIPEDRIVE_age"] > 30) & (dataframe["PESSOA_PIPEDRIVE_age"] <= 60)
dataframe["PESSOA_PIPEDRIVE ADULTO"].fillna(0)
dataframe["PESSOA_PIPEDRIVE ADULTO"].replace(True, 1, inplace=True)
dataframe["PESSOA_PIPEDRIVE ADULTO"].replace(False, 0, inplace=True)

dataframe["PESSOA_PIPEDRIVE IDOSO"] = dataframe["PESSOA_PIPEDRIVE_age"] > 60
dataframe["PESSOA_PIPEDRIVE IDOSO"].fillna(0)
dataframe["PESSOA_PIPEDRIVE IDOSO"].replace(True, 1, inplace=True)
dataframe["PESSOA_PIPEDRIVE IDOSO"].replace(False, 0, inplace=True)


dataframe["TWILIO_Ligações Outbound Qtd Significativa"] = dataframe["TWILIO_Ligações Outbound"] >= dataframe["TWILIO_Ligações Outbound"].mean()
dataframe["TWILIO_Ligações Outbound Qtd Significativa"].fillna(0)
dataframe["TWILIO_Ligações Outbound Qtd Significativa"].replace(True, 1, inplace=True)
dataframe["TWILIO_Ligações Outbound Qtd Significativa"].replace(False, 0, inplace=True)

dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"] = dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] == 0
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"].fillna(0)
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"].replace(True, 1, inplace=True)
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Nenhum"].replace(False, 0, inplace=True)

dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"] = (dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] > 0) & (dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] <= dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].mean() )
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"].fillna(0)
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"].replace(True, 1, inplace=True)
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Pouco"].replace(False, 0, inplace=True)

dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"] = dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] > dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].mean() 
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"].fillna(0)
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"].replace(True, 1, inplace=True)
dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia Muito"].replace(False, 0, inplace=True)

dataframe["ATENDIMENTOS_AGENDA_Qde Prescrições"].fillna(0,inplace=True)

dataframe = dataframe.drop(["ATENDIMENTOS_AGENDA_Datas Prescrição"],axis=1)

dataframe = dataframe[dataframe["PESSOA_PIPEDRIVE_id_gender"].isin([64,63])]   
dataframe["PESSOA_PIPEDRIVE_id_gender Binário"] = dataframe["PESSOA_PIPEDRIVE_id_gender"].map({64: 0, 63: 1})

dataframe = dataframe[dataframe["FUNIL_ASSINATURA_PIPEDRIVE_id_stage"].isin([65,64])]   
dataframe["FUNIL_ASSINATURA_PIPEDRIVE_id_stage Binário"] = dataframe["FUNIL_ASSINATURA_PIPEDRIVE_id_stage"].map({65: 0, 64: 1})



dataframe = pd.get_dummies(dataframe, columns=['PESSOA_PIPEDRIVE_id_marrital_status'], prefix='Status')
dataframe = pd.get_dummies(dataframe, columns=['PESSOA_PIPEDRIVE_state'], prefix='Estado')

ce = CountEncoder()
dataframe['PESSOA_PIPEDRIVE_city Codificada'] = ce.fit_transform(dataframe['PESSOA_PIPEDRIVE_city'])


dataframe = dataframe.drop(columns=["PESSOA_PIPEDRIVE_postal_code","PESSOA_PIPEDRIVE_id_person","PESSOA_PIPEDRIVE_city","PESSOA_PIPEDRIVE_id_gender","PESSOA_PIPEDRIVE_contract_start_date","PESSOA_PIPEDRIVE_contract_end_date","FUNIL_ASSINATURA_PIPEDRIVE_id_stage","FUNIL_ASSINATURA_PIPEDRIVE_id_org","FUNIL_ASSINATURA_PIPEDRIVE_lost_time","FUNIL_ASSINATURA_PIPEDRIVE_start_of_service","FUNIL_ONBOARDING_PIPEDRIVE_add_time","ATENDIMENTOS_AGENDA_Datas Atendimento Médico","ATENDIMENTOS_AGENDA_Datas Acolhimento","process_time"])


# feature engineering 2 + 3

dataframe = pd.get_dummies(dataframe,columns=["FUNIL_ASSINATURA_PIPEDRIVE_status"], prefix='status')


dataframe = pd.get_dummies(dataframe,columns=['FUNIL_ASSINATURA_PIPEDRIVE_lost_reason'], prefix='lost_reason')

dataframe = pd.get_dummies(dataframe,columns=['PESSOA_PIPEDRIVE_Canal de Preferência'], prefix='canal_preferencia')    

dataframe = pd.get_dummies(dataframe,columns=['FUNIL_ONBOARDING_PIPEDRIVE_status'], prefix='Status')


dataframe = pd.get_dummies(dataframe,columns=['FUNIL_ONBOARDING_PIPEDRIVE_lost_reason'], prefix='lost_reason')


dataframe.drop('ATENDIMENTOS_AGENDA_Qde Todos Atendimentos', axis='columns', inplace=True)



# feature engineering 4 + 3

if dataframe['WHOQOL_Físico_New'].dtype == 'object':
    # Aplicando codificação one-hot para variáveis categóricas
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(dataframe[['WHOQOL_Físico_New']])
    encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out(['WHOQOL_Físico_New']))
    dataframe = dataframe.join(encoded_df)
else:
    # Aplicando normalização ou padronização para variáveis numéricas
    scaler = StandardScaler()
    if dataframe['WHOQOL_Físico_New'].count() >= 2:
        dataframe['WHOQOL_Físico_New'] = scaler.fit_transform(dataframe[['WHOQOL_Físico_New']])
    else:
        # Lide com o caso de ter apenas uma linha ou todos os valores NaN
        dataframe['WHOQOL_Físico_New'] = dataframe['WHOQOL_Físico_New'].fillna(0)

# Calculando o intervalo interquartil
Q1 = dataframe['WHOQOL_Físico_New'].quantile(0.25)
Q3 = dataframe['WHOQOL_Físico_New'].quantile(0.75)
IQR = Q3 - Q1

# Removendo os outliers
dataframe = dataframe[(dataframe['WHOQOL_Físico_New'] >= Q1 - 1.5 * IQR) & (dataframe['WHOQOL_Físico_New'] <= Q3 + 1.5 * IQR)]



# Verificando se é categórica ou numérica
if dataframe['WHOQOL_Psicológico_New'].dtype == 'object':
    # Aplicando codificação one-hot para variáveis categóricas
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(dataframe[['WHOQOL_Psicológico_New']])
    encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out(['WHOQOL_Psicológico_New']))
    dataframe = dataframe.join(encoded_df)
else:
    # Aplicando normalização ou padronização para variáveis numéricas
    scaler = StandardScaler()
    if dataframe['WHOQOL_Psicológico_New'].count() >= 2:
        dataframe['WHOQOL_Psicológico_New'] = scaler.fit_transform(dataframe[['WHOQOL_Psicológico_New']])
    else:
        # Lide com o caso de ter apenas uma linha ou todos os valores NaN
        dataframe['WHOQOL_Psicológico_New'] = dataframe['WHOQOL_Psicológico_New'].fillna(0)

    # Calculando o intervalo interquartil
    Q1 = dataframe['WHOQOL_Psicológico_New'].quantile(0.25)
    Q3 = dataframe['WHOQOL_Psicológico_New'].quantile(0.75)
    IQR = Q3 - Q1

    # Removendo os outliers
    dataframe = dataframe[(dataframe['WHOQOL_Psicológico_New'] >= Q1 - 1 * IQR) & (dataframe['WHOQOL_Psicológico_New'] <= Q3 + 1 * IQR)]

# Verificando se é categórica ou numérica
if dataframe['WHOQOL_Social_New'].dtype == 'object':
    # Aplicando codificação one-hot para variáveis categóricas
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(dataframe[['WHOQOL_Social_New']])
    encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out(['WHOQOL_Social_New']))
    data = data.join(encoded_df)
else:
    # Aplicando normalização ou padronização para variáveis numéricas
    scaler = StandardScaler()
    if dataframe['WHOQOL_Social_New'].count() >= 2:
        dataframe['WHOQOL_Social_New'] = scaler.fit_transform(dataframe[['WHOQOL_Social_New']])
    else:
        # Lide com o caso de ter apenas uma linha ou todos os valores NaN
        dataframe['WHOQOL_Social_New'] = dataframe['WHOQOL_Social_New'].fillna(0)

Q1 = dataframe['WHOQOL_Social_New'].quantile(0.25)
Q3 = dataframe['WHOQOL_Social_New'].quantile(0.75)
IQR = Q3 - Q1

# Removendo os outliers
dataframe = dataframe[(dataframe['WHOQOL_Social_New'] >= Q1 - 1 * IQR) & (dataframe['WHOQOL_Social_New'] <= Q3 + 1 * IQR)]

dataframe = pd.get_dummies(dataframe, columns=['last_stage_concluded'], prefix='stage')

dataframe.to_csv('../notebooks/data/data-engineering.csv', index=False)

In [173]:
df2 = pd.read_csv("../notebooks/data/data-preprocessed.csv")

df2.head()

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_contract_start_date,PESSOA_PIPEDRIVE_contract_end_date,PESSOA_PIPEDRIVE_Canal de Preferência,PESSOA_PIPEDRIVE_notes_count,...,WHOQOL_Psicológico_New,WHOQOL_Social_New,WHOQOL_Ambiental_New,COMUNICARE_Problemas Abertos Bool,TWILIO_Data Última Mensagens Inbound Recente,stay_time,last_stage_concluded,process_time,TWILIO_Data Última Mensagens Outbound Recente,TWILIO_Data Última Ligações Outbound Recente
0,1080,63,80,Pernambuco,Recife,50070-045,2021-07-19,2023-08-18,0.0,5,...,1.7,4.0,3.5,1,0,737 days,Questionário,2022-04-18 15:07:13,0,0


In [174]:
import pandas as pd
df = pd.read_csv("../notebooks/data/data-engineering.csv")

In [175]:
data = pd.DataFrame(df.iloc[50])

IndexError: single positional indexer is out-of-bounds

In [ ]:
preprocessing(data)

NameError: name 'preprocessing' is not defined

In [ ]:
df.head()

,PESSOA_PIPEDRIVE_notes_count,PESSOA_PIPEDRIVE_done_activities_count,FUNIL_ONBOARDING_PIPEDRIVE_activities_count,ATENDIMENTOS_AGENDA_Faltas Todos Atendimento,ATENDIMENTOS_AGENDA_Qde Atendimento Médico,ATENDIMENTOS_AGENDA_Faltas Atendimento Médico,ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento,ATENDIMENTOS_AGENDA_Faltas Acolhimento,ATENDIMENTOS_AGENDA_Qde Psicoterapia,ATENDIMENTOS_AGENDA_Qde Prescrições,...,FUNIL_ASSINATURA_PIPEDRIVE_id_stage Binário,Status_80,Estado_Santa Catarina,PESSOA_PIPEDRIVE_city Codificada,status_lost,lost_reason_Outro,canal_preferencia_0,Status_lost,lost_reason_Outro.1,stage_Boas-vindas
0,0,0,0,0,0,0,0,0,0,0.0,...,0,True,True,1,True,True,True,True,True,True


In [ ]:
df_pimba = preprocessing(df)


DEU RUIM


In [ ]:
df = pd.read_csv("data-engineering.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data-engineering.csv'

In [ ]:
df["FUNIL_ASSINATURA_PIPEDRIVE_id_stage"].value_counts()

KeyError: 'FUNIL_ASSINATURA_PIPEDRIVE_id_stage'

In [ ]:
df["PESSOA_PIPEDRIVE_Canal de Preferência"].value_counts()

PESSOA_PIPEDRIVE_Canal de Preferência
0      571
239    379
238     12
360     11
Name: count, dtype: int64

In [ ]:
df.iloc[:,60:].head()

,Estado_Mato Grosso do Sul,Estado_Minas Gerais,Estado_Paraná,Estado_Paraíba,Estado_Pará,Estado_Pernambuco,Estado_Piauí,Estado_Rio Grande do Norte,Estado_Rio Grande do Sul,Estado_Rio de Janeiro,Estado_Santa Catarina,Estado_Sergipe,Estado_State of Amazonas,Estado_São Paulo,PESSOA_PIPEDRIVE_city Codificada,status_lost,status_won,lost_reason_Outro,lost_reason_[Assinatura] Cancelamento por inadimplência,lost_reason_[Assinatura] Desligamento,lost_reason_[Assinatura] Empresa cancelou o benefício da Ana,lost_reason_[Assinatura] Está sem tempo para conciliar os atendimentos,lost_reason_[Assinatura] Não quer seguir com a Ana,lost_reason_[Assinatura] Precisou cortar custos,Status_Não iniciado,Status_lost,Status_open,Status_won,lost_reason_Outro.1,lost_reason_[Associade] Cancelou assinatura,lost_reason_[Onboarding] Não retornou aos contatos de resgate,lost_reason_[Onboarding] Não tem interesse em seguir nas etapas do onboarding,stage_Boas-vindas,stage_Não iniciado,stage_Primeira reunião,stage_Questionário
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,32,True,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,True
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,32,True,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,True
2,False,True,False,False,False,False,False,False,False,False,False,False,False,False,3,True,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,False,False,True,5,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1,True,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True


In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import datetime as dt

def column_label_to_index(col_label):
    col_index = 0
    for c in col_label.upper():
        col_index = col_index * 26 + (ord(c) - ord('A') + 1)
    return col_index - 1

def add_prefix_to_columns(dataframe, prefix):
    for col in dataframe.columns:
        new_col_name = prefix + str(col)
        dataframe.rename(columns={col: new_col_name}, inplace=True)


def mergeHeader_Columns(data):
    add_prefix_to_columns(data, 'PESSOA_PIPEDRIVE_')
    add_prefix_to_columns(data, 'FUNIL_ASSINATURA_PIPEDRIVE_')
    add_prefix_to_columns(data, 'FUNIL_ONBOARDING_PIPEDRIVE_')
    add_prefix_to_columns(data, 'ATENDIMENTOS_AGENDA_')
    add_prefix_to_columns(data, 'WHOQOL_')
    add_prefix_to_columns(data, 'COMUNICARE_')
    add_prefix_to_columns(data, 'TWILIO_')
    add_prefix_to_columns(data, 'COBRANÇA_VINDI_')

    return data

def preprocessing(data_dataframe):
    from sklearn.impute import SimpleImputer

    # try:
    data_dataframe = mergeHeader_Columns(data_dataframe)
    data_dataframe = pd.DataFrame(data_dataframe.iloc[50])
    # preprocessing 1
    data_dataframe = data_dataframe.drop(["PESSOA_PIPEDRIVE_id_person_recommendation","PESSOA_PIPEDRIVE_Recebe Comunicados?", "PESSOA_PIPEDRIVE_Interesses", "PESSOA_PIPEDRIVE_Pontos de Atenção", "FUNIL_ONBOARDING_PIPEDRIVE_id_label"], axis=1)

    data_dataframe['PESSOA_PIPEDRIVE_birthdate'] = pd.to_datetime(data_dataframe['PESSOA_PIPEDRIVE_birthdate'])
    data_dataframe['PESSOA_PIPEDRIVE_age'] = data_dataframe['PESSOA_PIPEDRIVE_birthdate'].apply(
        lambda x: datetime.today().year - x.year - ((datetime.today().month, datetime.today().day) < (x.month, x.day))
    )

    imputer = SimpleImputer(strategy='mean')
    data_dataframe['PESSOA_PIPEDRIVE_age'] = imputer.fit_transform(data_dataframe['PESSOA_PIPEDRIVE_age'].values.reshape(-1, 1))
    data_dataframe['PESSOA_PIPEDRIVE_age'] = np.round(data_dataframe['PESSOA_PIPEDRIVE_age']).astype(int)

    data_dataframe = data_dataframe.drop(['PESSOA_PIPEDRIVE_birthdate'], axis=1 )

    rows_to_drop = data_dataframe[data_dataframe["PESSOA_PIPEDRIVE_id_gender"].isin([117,110,111])]
    data_dataframe = data_dataframe.drop(rows_to_drop.index, axis=0)
    data_dataframe["PESSOA_PIPEDRIVE_id_gender"].fillna(64, inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_id_marrital_status"].fillna(80, inplace=True)

    
    data_dataframe["PESSOA_PIPEDRIVE_state"].fillna(data_dataframe["PESSOA_PIPEDRIVE_state"].mode()[0], inplace=True)


    data_dataframe["PESSOA_PIPEDRIVE_city"].fillna(data_dataframe["PESSOA_PIPEDRIVE_city"].mode()[0], inplace=True)


    

    if data_dataframe.shape[0] == 1 and data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].isna().any():
        data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].fillna(data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].mode()[0], inplace=True)
    data_dataframe['PESSOA_PIPEDRIVE_has_public_health_plan'] = data_dataframe['PESSOA_PIPEDRIVE_id_health_plan'].apply(lambda x: 1 if int(x) == 412 else 0)
    data_dataframe = data_dataframe.drop(['PESSOA_PIPEDRIVE_id_health_plan'], axis=1)
    data_dataframe["PESSOA_PIPEDRIVE_tem_data_dataframe_de_termino_de_contrato"] = data_dataframe["PESSOA_PIPEDRIVE_contract_end_date"].apply(lambda x: 0 if pd.isna(x) else 1)
    data_dataframe.drop(["PESSOA_PIPEDRIVE_id_continuity_pf"], axis=1, inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_Canal de Preferência"].fillna(0, inplace=True)
    data_dataframe["PESSOA_PIPEDRIVE_Tem_Canal_de_Preferência"] = data_dataframe["PESSOA_PIPEDRIVE_Canal de Preferência"].apply(lambda x: 1 if int(x) > 0  else 0)
    data_dataframe["PESSOA_PIPEDRIVE_has_notes"] = data_dataframe["PESSOA_PIPEDRIVE_notes_count"].apply(lambda x: 1 if int(x) > 0 else 0)
    
    # preprocessing 2
    data_dataframe = data_dataframe.drop(["ATENDIMENTOS_AGENDA_Faltas Psicoterapia","TWILIO_Ligações Inbound", "TWILIO_Data Última Ligações Inbound","COBRANÇA_VINDI_Qde Total de Faturas","COBRANÇA_VINDI_Qde Total de Tentativas de Cobrança","COBRANÇA_VINDI_Método de Pagamento","COBRANÇA_VINDI_Valor Médio da Mensalidade","COBRANÇA_VINDI_Qde Total de Faturas Pagas após Vencimento","COBRANÇA_VINDI_Qde Total de Faturas Inadimpletes","COBRANÇA_VINDI_Valor Total Inadimplência"], axis=1)

    data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].fillna(0, inplace=True)
    


    data_dataframe = data_dataframe.drop(columns="WHOQOL_Qde Respostas WHOQOL")

    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico"].str.split(';').str[-1].str.strip()

    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].str.replace(",",".")

    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].astype(float) 

    data_dataframe["WHOQOL_Físico_New"] = data_dataframe["WHOQOL_Físico_New"].fillna(data_dataframe["WHOQOL_Físico_New"].median())

    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico"].str.split(';').str[-1].str.strip()

    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].str.replace(",",".")

    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].astype(float) 

    data_dataframe["WHOQOL_Psicológico_New"] = data_dataframe["WHOQOL_Psicológico_New"].fillna(data_dataframe["WHOQOL_Psicológico_New"].median())

    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social"].str.split(';').str[-1].str.strip()

    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].str.replace(",",".")

    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].astype(float) 

    data_dataframe["WHOQOL_Social_New"] = data_dataframe["WHOQOL_Social_New"].fillna(data_dataframe["WHOQOL_Social_New"].median())

    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental"].str.split(';').str[-1].str.strip()

    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].str.replace(",",".")

    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].astype(float) 

    data_dataframe["WHOQOL_Ambiental_New"] = data_dataframe["WHOQOL_Ambiental_New"].fillna(data_dataframe["WHOQOL_Ambiental_New"].median())

    data_dataframe["COMUNICARE_Problemas Abertos Bool"] = data_dataframe["COMUNICARE_Problemas Abertos"].notnull().astype(int)

    data_dataframe["COMUNICARE_Problemas Abertos Bool"].fillna(0, inplace=True)

    data_dataframe["TWILIO_Mensagens Inbound"].fillna(0, inplace=True)

    data_dataframe["TWILIO_Data Última Mensagens Inbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Mensagens Inbound"])

    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Mensagens Inbound"]

    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].fillna('', inplace=True)

    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].astype(str)

    data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].str.extract('(\d+) days').astype(float)

    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"] = data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"] < data_dataframe["TWILIO_Data Última Mensagens Inbound Tempo Passado"].median()
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].fillna(0)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].replace(True, 1, inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Inbound Recente"].replace(False, 0, inplace=True)

    data_dataframe = data_dataframe.drop(columns=["WHOQOL_Ambiental","WHOQOL_Social","WHOQOL_Físico","WHOQOL_Psicológico","COMUNICARE_Problemas Abertos","TWILIO_Data Última Mensagens Inbound","ATENDIMENTOS_AGENDA_Datas Psicoterapia","TWILIO_Data Última Mensagens Inbound Tempo Passado"])
    
    # preprocessing 3
    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        if pd.notna(valor) == False: 
            if pd.notna(data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_end_date"]):
                data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_end_date"]

    data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].fillna(dt.date.today(), inplace=True)

    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        index = str(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"]).find(";")
        if index != -1:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"][:index]

    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        tamanho = len(str(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"]))
        if tamanho > 10:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"][:10]	

    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = pd.to_datetime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], format='%Y-%m-%d', errors='coerce')
        data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].strftime('%Y-%m-%d')

    tempo_permanencia = []

    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].items():
        if pd.notna(valor):
            index = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"].find(";")
            if index != -1:
                data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"][:index]

    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_time"].items():
        if pd.notna(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"]):
            tempo_1 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
            tempo_2 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_start_of_service"], '%Y-%m-%d')
            tempo_permanencia.append(str(tempo_1 - tempo_2))
        else:
            tempo_1 = datetime.strptime(data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_time"], '%Y-%m-%d')
            tempo_2 = datetime.strptime(data_dataframe.loc[indice, "PESSOA_PIPEDRIVE_contract_start_date"], '%Y-%m-%d')
            tempo_permanencia.append(str(tempo_1 - tempo_2))

    data_dataframe['stay_time'] = tempo_permanencia

    for indice, valor in data_dataframe["stay_time"].items():
        index = data_dataframe.loc[indice, "stay_time"].find(",")
        if index != -1:
            data_dataframe.loc[indice, "stay_time"] = data_dataframe.loc[indice, "stay_time"][:index]

    for indice, valor in data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].items():
        if pd.notna(valor):  
            index = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].find(";")
            if index != -1: 
                data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"][:index]
        else:
            data_dataframe.loc[indice, "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = "Assinatura ativa"
            
        contagem = data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].value_counts()

        agrupamento = contagem[contagem < 20].index
        data_dataframe.loc[data_dataframe["FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"].isin(agrupamento), "FUNIL_ASSINATURA_PIPEDRIVE_lost_reason"] = "Outro"

    data_dataframe['FUNIL_ONBOARDING_PIPEDRIVE_add_time'].fillna('Não iniciado', inplace=True)

    ultimos_estados = []

    for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
        if valor != "Não iniciado":
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome"]):
                if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting"]):
                    if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"]):
                        ultimos_estados.append("Questionário")
                    else:
                        ultimos_estados.append("Primeira reunião")
                else:
                    ultimos_estados.append("Boas-vindas")
            
        else:
            ultimos_estados.append("Não iniciado")

    data_dataframe["last_stage_concluded"] = ultimos_estados

    data_dataframe.drop(["FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_welcome", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_first_meeting", "FUNIL_ONBOARDING_PIPEDRIVE_stay_in_pipeline_stages_whoqol"], axis=1, inplace=True)

    data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_status"].fillna("Não iniciado", inplace=True)

    tempo = []

    for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_add_time"].items():
        if valor != "Não iniciado":
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"]) == True:
                tempo.append(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"])
            elif pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"]) == True:
                tempo.append(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_won_time"])
            else:
                tempo.append("Em aberto")
        else:
            tempo.append("Não iniciado")

    data_dataframe['process_time'] = tempo

    data_dataframe.drop(["FUNIL_ONBOARDING_PIPEDRIVE_won_time", "FUNIL_ONBOARDING_PIPEDRIVE_lost_time"], axis=1, inplace=True)

    data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_activities_count"] = data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_activities_count"].fillna(0)

    data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimento Médico"].fillna(0)

    data_dataframe["ATENDIMENTOS_AGENDA_Faltas Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Faltas Atendimento Médico"].fillna(0)

    data_dataframe["ATENDIMENTOS_AGENDA_Datas Atendimento Médico"] = data_dataframe["ATENDIMENTOS_AGENDA_Datas Atendimento Médico"].fillna("Nunca ocorreu")

    for indice, valor in data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].items():
        if pd.notna(valor) == False:
            if pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_status"]) == False and pd.notna(data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_add_time"]):
                data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo em aberto"
            if data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_status"] == "won":
                data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo concluído"
            else:
                data_dataframe.loc[indice, "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Processo não iniciado"

        contagem = data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].value_counts()

        agrupamento = contagem[contagem < 23].index
        data_dataframe.loc[data_dataframe["FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"].isin(agrupamento), "FUNIL_ONBOARDING_PIPEDRIVE_lost_reason"] = "Outro"

    data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Atendimentos Acolhimento"].fillna(0)

    data_dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Faltas Acolhimento"].fillna(0)

    data_dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"] = data_dataframe["ATENDIMENTOS_AGENDA_Datas Acolhimento"].fillna("Nunca ocorreu")

    data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"] = data_dataframe["ATENDIMENTOS_AGENDA_Qde Psicoterapia"].fillna(0)

    # preprocessing 4

    data_dataframe["TWILIO_Mensagens Outbound"].fillna(0, inplace=True)

    data_dataframe["TWILIO_Data Última Mensagens Outbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Mensagens Outbound"])

    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Mensagens Outbound"]

    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].fillna('', inplace=True)
    
    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].astype(str)

    data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].str.extract('(\d+) days').astype(float)

    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Mensagens Outbound"], axis=1)

    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"] = data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"] < data_dataframe["TWILIO_Data Última Mensagens Outbound Tempo passado"].median()
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].fillna(0)
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].replace(True, 1, inplace=True)
    data_dataframe["TWILIO_Data Última Mensagens Outbound Recente"].replace(False, 0, inplace=True)

    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Mensagens Outbound Tempo passado"], axis=1)

    data_dataframe["TWILIO_Ligações Outbound"].fillna(0, inplace=True)

    data_dataframe["TWILIO_Data Última Ligações Outbound"] = pd.to_datetime(data_dataframe["TWILIO_Data Última Ligações Outbound"])

    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = datetime.now() - data_dataframe["TWILIO_Data Última Ligações Outbound"]

    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].fillna('', inplace=True)

    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].astype(str)

    data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].str.extract('(\d+) days').astype(float)

    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Ligações Outbound"], axis=1)

    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"] = data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"] < data_dataframe["TWILIO_Data Última Ligações Outbound Tempo passado"].median()
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].fillna(0)
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].replace(True, 1, inplace=True)
    data_dataframe["TWILIO_Data Última Ligações Outbound Recente"].replace(False, 0, inplace=True)

    data_dataframe = data_dataframe.drop(["TWILIO_Data Última Ligações Outbound Tempo passado"], axis=1)

    data_dataframe = data_dataframe.drop(["COBRANÇA_VINDI_Qde Perfis de Pagamento Inativos"], axis=1)

    # preprocessing especials 

    data_dataframe = data_dataframe[data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'].isin(['won', 'lost'])]

    data_dataframe = data_dataframe.drop(data_dataframe[(data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'] == 'won') & (~data_dataframe['PESSOA_PIPEDRIVE_contract_end_date'].isnull())].index)

    data_dataframe = data_dataframe.drop(data_dataframe[(data_dataframe['FUNIL_ASSINATURA_PIPEDRIVE_status'] == 'lost') & (data_dataframe['PESSOA_PIPEDRIVE_contract_end_date'].isnull())].index)

    #exportando df pronto

    data_dataframe.to_csv('../notebooks/data/data-preprocessed.csv', index=False)

    #     return data_dataframe
    # except Exception:
    #     print("DEU RUIM")
    #     return False


In [ ]:
data = pd.read_csv("../notebooks/data/Ana Health_Tabela Modelo Previsão Churn - Tabela.csv")

In [ ]:
data_merge.head()

,COBRANÇA_VINDI_TWILIO_COMUNICARE_WHOQOL_ATENDIMENTOS_AGENDA_FUNIL_ONBOARDING_PIPEDRIVE_FUNIL_ASSINATURA_PIPEDRIVE_PESSOA_PIPEDRIVE_50
"PESSOA, PIPEDRIVE",1022
Unnamed: 1,1993-08-03
Unnamed: 2,64
Unnamed: 3,80
Unnamed: 4,Santa Catarina


In [ ]:
preprocessing(data)

KeyError: "['PESSOA_PIPEDRIVE_id_person_recommendation', 'PESSOA_PIPEDRIVE_Recebe Comunicados?', 'PESSOA_PIPEDRIVE_Interesses', 'PESSOA_PIPEDRIVE_Pontos de Atenção', 'FUNIL_ONBOARDING_PIPEDRIVE_id_label'] not found in axis"